In [1]:
!pip install openai
import requests
import random
import re
import ast
from openai import OpenAI
from google.colab import userdata
API_key = userdata.get('openai')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00


##UTILS

In [13]:
#@title Game Translator
def translator(lang, prompt, model = "gpt-4o-mini"):
  client = OpenAI(api_key=API_key)
  chat_completion = client.chat.completions.create(
        model=model,
        messages=[
                  {"role": "system", "content": f"you are a useful TRANSLATOR from english to {lang}. Don't add punctuation where is not required. We are playing the board game CODENAMES and you need to supply a precise translation of the following prompt: "},
                  {"role": "user", "content": prompt}
                  ]
    )
  return chat_completion.choices[0].message.content

def board_translator(lang, prompt, model = "gpt-4o-mini"):
  client = OpenAI(api_key=API_key)

  system_prompt = "You are a helpful assistant designed to output JSON. (DICT WITH KEYS: word_to_translate, translated_word)" + \
    f"You are a professional translator. Your task is to translate a list of strings of text from English into {lang}." + \
    "Ensure the translations are accurate and contextually appropriate. Maintain the original meaning and tone of the text." + \
    "Use the following format for your response. The input you are :" + \
    "Example:" + \
    "Original Text: ['Hello', 'World', ... , 'Room']"+ \
    "Your response: {'Hello':'Ciao', 'World':'Mondo', ... , 'Room':'Stanza'}"

  chat_completion = client.chat.completions.create(
        model=model,
        response_format={ "type": "json_object" },
        messages=[
                  {"role": "system", "content": system_prompt },
                  {"role": "user", "content": prompt}
                  ]
    )
  return chat_completion.choices[0].message.content

In [25]:
#@title board utils
def generate_board(n = 25, lang = "english", c = 7, k = 1, model = "gpt-4o-mini"):
  '''
  inputs:
    n (int): number of words
    lang (str): language of words
    c (int): number of colored words for each team
    k (int): number of killer words

  returns a dictionary such that dict[word] = color
  '''
  blue_cards = c
  red_cards = c + 1
  if (2*c + k + 1) > n:
    raise Exception("ERROR: wrong parameters for board creation")

  url = f"https://raw.githubusercontent.com/mich1803/Codenames-LLM/main/wordlists/eng.txt"

  try:
    # Fetch the content of the file
    response = requests.get(url)
  except:
    raise Exception("ERROR: The language entered is wrong or has no dictionary in the github folder")
  words = response.text.splitlines()

  # Select n unique random words
  random_words = random.sample(words, n)
  if lang != "english":
    tr = board_translator(lang, "[" + ",".join(random_words) + "]")
    tr = ast.literal_eval(tr)
    tr_random_words = [tr[word] for word in random_words]
    random_words = tr_random_words

  random_words = [word.upper() for word in random_words]

  # Select indices for the colors
  indices = list(range(n))
  blue_indices = random.sample(indices, blue_cards)
  remaining_indices = [i for i in indices if i not in blue_indices]
  red_indices = random.sample(remaining_indices, red_cards)
  remaining_indices = [i for i in remaining_indices if i not in red_indices]
  black_index = random.sample(remaining_indices, k)

  colors = [
      "BLUE" if i in blue_indices else
      "RED" if i in red_indices else
      "KILLER" if i in black_index else
      "WHITE"
      for i in range(n)
      ]

  word_color_dict = {random_words[i]: colors[i] for i in range(n)}

  return word_color_dict

def board4prompt(word_color_dict, master = False):
  '''
  inputs:
    word_color_dict(dict): board dict generated from generate_board()

  return a string version for the prompt for LLMs
  '''

  text = "|"
  if master:
    for idx, i in enumerate(word_color_dict):
      text += f" {i} ({word_color_dict[i]}) " + "|"
      if (idx > 0) and (((idx + 1) % 5) == 0):
        text += "\n"
        if idx != (len(word_color_dict)-1):
          text += "|"
  else:
    for idx, i in enumerate(word_color_dict):
      text += f" {i} " + "|"
      if (idx > 0) and ((idx % 5) == 0):
        text += "\n"
        if idx != (len(word_color_dict)-1):
          text += "|"


  return text

## GAME (1 SPYMASTER, k GUESSERS, 1 CAPTAIN GUESSER)

In [37]:
prompt_colors = {
    "RED": "\033[1;31m",
    "BLUE": "\033[1;34m",
    "endcolor": "\033[0m",
    "n": "\033[1;33m"
  }

#@title agents definition
def spymaster_agent(lang, board, team, model = "gpt-4o-mini"):
    systemprompt = "We are playing the board game CODENAMES, and your role will be the spymaster. \n" + \
      f"Given a list of words, each with an associated color: RED for team red words, BLUE for team blue, KILLER for killer words and NEUTRAL for neutral words. \n You are part of the team {team}. You need to help your team's guessers find only the words with your team's color. \n" + \
      f"If your team guess the word correctly, you gain 1 point; if your team guess an opponent team word, they gain 1 point; if your team guess the killer word you lose instantly so play CAREFULLY. \n" + \
      "YOU NEED TO PROVIDE THEM WITH A ONE-WORD CLUE (that is not on the board) and A NUMBER that represents the number of words on the board related to that clue. \n"

    if lang != "english":
      systemprompt = translator(lang, systemprompt)

    client = OpenAI(api_key=API_key)
    chat_completion = client.chat.completions.create(
        model=model,
        response_format={ "type": "json_object" },
        messages=[
                  {"role": "system", "content": "*You are a helpful assistant designed to output JSON. (DICT WITH KEYS: clue, number)* \n" + systemprompt},
                  {"role": "user", "content": board}
                  ]
    )
    return chat_completion.choices[0].message.content


def guesser_agent(lang, team, board, clue, number, n_guessers, i_agent, cards_remaining, k, prev = False,  model = "gpt-4o-mini"):
    """
    lang -> language of the game
    board -> actual board
    clue -> clue from the spymaster
    n_guessers -> number of guessers
    i_agent -> index of the agent
    allied_cards -> number of red cards
    enemy_cards -> number of blue cards
    k -> number of killer words
    prev -> previous agents thoughts

    outputs thoughts and strategies for other guessers
    """
    opp = "BLUE" if team == "RED" else "RED"

    systemprompt = "We are playing the board game CODENAMES, and your role will be the guesser. \n" + \
      f"You are on a team of {n_guessers} other guessers. Your objective as a team is to guess the words of your color based on the clue of your spymaster \n" + \
      f"There are: {cards_remaining[team]} words for your team, {cards_remaining[opp]} words for the enemy team and {k} killer cards." + \
      f"Even if the number said by the spymaster is more than one, let's guess one word at a time."

    p1 = "The board is the following: \n"
    p2 = "\n" + "The clue is: "
    p3 = "\n" + "The previous thoughts are: \n"
    p4 = "\n" + "The number is: \n"
    if lang != "english":
      p1 = translator(lang, p1)
      p2 = translator(lang, p2)
      p3 = translator(lang, p3)
      p4 = translator(lang, p4)

    if i_agent == 1:
      systemprompt += "As the first guesser you need to provide thoughts and strategies for your fellow guessers only reading the board and the clue from the spymaster." + \
      "Breakdown the problem in subproblems if you need to."
      prompt = p1 + board + p2 + clue + p4 + str(number) + "\n"
    else:
      systemprompt += "You need to provide thoughts and strategies for your fellow guessers reading the board, the clue from the spymaster and the thoghts of your previous fellow guessers." + \
      "If you don't think that those thoughts are correct feel free to contradict them. Breakdown the problem in subproblems if you need to."
      prompt = p1 + board + p2 + clue + p4 + str(number) + p3 + prev

    m = [{"role": "system", "content": systemprompt},
        {"role": "user", "content": prompt}]

    if lang != "english":
      systemprompt = translator(lang, systemprompt)

    client = OpenAI(api_key=API_key)
    chat_completion = client.chat.completions.create(
        model=model,
        messages = m
      )

    return chat_completion.choices[0].message.content


def captain_guesser_agent(lang, team, board, clue, cards_remaining, k, prev,  model = "gpt-4o-mini"):
    opp = "BLUE" if team == "RED" else "RED"
    systemprompt = "We are playing the board game CODENAMES, and your role will be the captain guesser. \n" + \
      f"You are on a team of other guessers. Your objective as a team is to guess the words of your color based on the clue of your spymaster \n" + \
      f"There are: {cards_remaining[team]} words for your team, {cards_remaining[opp]} words for the enemy team and {k} killer cards." + \
      "As the captain guesser you need to read thoughts and strategies from your fellows guesser and then guess a SINGLE WORD from the board that you think is right."

    p1 = "The board is the following: \n"
    p2 = "\n" + "The clue is: "
    p3 = "\n" + "Your fellows' thoughts are: \n"

    if lang != "english":
      systemprompt = translator(lang, systemprompt)
      p1 = translator(lang, p1)
      p2 = translator(lang, p2)
      p3 = translator(lang, p3)

    prompt = p1 + board + p2 + clue + p3 + prev

    client = OpenAI(api_key=API_key)
    chat_completion = client.chat.completions.create(
        model=model,
        response_format={ "type": "json_object" },
        messages=[
                  {"role": "system", "content": "*You are a helpful assistant designed to output JSON. (DICT WITH KEY: guess)* \n" + systemprompt},
                  {"role": "user", "content": prompt}
                  ]
    )
    return chat_completion.choices[0].message.content



def read_clue(answer):
  try:
    data = ast.literal_eval(answer)
    return (data["clue"].upper(), data["number"])
  except:
    raise ValueError("Format error")

def read_guess(answer):
  try:
    data = ast.literal_eval(answer)
    if type(data["guess"]) == list:
      guess = data["guess"][0].upper()
      return guess
    else:
      return data["guess"].upper()
  except:
    raise ValueError("Format error")





In [35]:
#@title play turn function
def play_turn(lang, team, board, cards_remaining, k, n_guessers, model, verbose):
  opp = "BLUE" if team == "RED" else "RED"
  rc = cards_remaining
  b = board

  if verbose:
    print(prompt_colors["n"] + "NARRATOR:" + prompt_colors["endcolor"] + " " + prompt_colors[team] + team + " team " + prompt_colors["endcolor"] + "turn, \n")
    print("the actual board is: \n")
    print(board4prompt(b, master = True))
    print("\n\n")

  clue, number = read_clue(spymaster_agent(lang, board4prompt(b, master = True), team, model))

  if verbose:
    print(prompt_colors[team] + " " + team + " SPYMASTER:" + prompt_colors["endcolor"] + f"clue: {clue}, number: {number}")
    print("\n\n")

  if clue in b:
    return {"endgame": True, "win": opp, "reason": "spymaster said a word in the board"}

  m = ""
  for i in range(number):
    for j in range(n_guessers):
      m += f"{team} guesser {j+1} said: {guesser_agent(lang, team, board4prompt(b), clue, number-i, n_guessers, j+1, cards_remaining, k, m, model)} \n \n"
    if verbose:
      print(m)
      print("\n\n")

    guess = read_guess(captain_guesser_agent(lang, team, board4prompt(b), clue, cards_remaining, k, m, model))
    if verbose:
      print(prompt_colors[team] + team + "CAPTAIN GUESSER:" + prompt_colors["endcolor"] + f"guess: {guess}")

    try:
        x = b[guess]
    except:
        if verbose: print(f"{prompt_colors['n']}NARRATOR{prompt_colors['endcolor']}: CAPTAIN GUESSER({team})'s guess wasn't on the board. \n \n")
        return {"endgame": False, "rc": rc, "b": b}

    if x == "KILLER":
        if verbose: print(f"{prompt_colors['n']}NARRATOR{prompt_colors['endcolor']}: The killer word have been selected, the game ends. \n \n ")
        return {"endgame": True, "win": opp, "reason": "guessed killer word"}

    elif x == team:
        rc[team] -= 1
        if verbose: print(f'{prompt_colors["n"]}NARRATOR{prompt_colors["endcolor"]}: A {prompt_colors[team]}{team} word{prompt_colors["endcolor"]} have been selected ({team} cards remaining = {rc[team]}). \n \n')
        del b[guess]
        if rc[team] == 0:
            if verbose: print(f"{prompt_colors['n']}NARRATOR{prompt_colors['endcolor']}: The {prompt_colors[team]}{team} team{prompt_colors['endcolor']} reached the goal, the game ends. \n \n")
            return {"endgame": True, "win": team, "reason": "cards finished"}

    elif x == opp:
        rc[opp] -= 1
        if verbose: print(f'{prompt_colors["n"]}NARRATOR{prompt_colors["endcolor"]}: A {prompt_colors[opp]}{opp} word{prompt_colors["endcolor"]} have been selected ({opp} cards remaining = {rc[opp]}). \n \n')
        del b[guess]
        if rc[opp] == 0:
            if verbose: print(f"{prompt_colors['n']}NARRATOR{prompt_colors['endcolor']}: The {prompt_colors[opp]}{opp} team{prompt_colors['endcolor']} reached the goal, the game ends. \n \n")
            return {"endgame": True, "win": opp, "reason": "cards finished"}
        break

    else:
        if verbose: print(f"{prompt_colors['n']}NARRATOR{prompt_colors['endcolor']}: A neutral word have been selected. \n \n")
        del b[guess]
        break

  return {"endgame": False, "rc": rc, "b": b}

In [10]:
#@title play game function
def play_game(lang = "english", n_cards = 25, coloured_cards = 7, k_cards = 1, verbose = False, red_model = "gpt-4o-mini", blue_model = "gpt-4o-mini", red_agents = 3, blue_agents = 3):
  r = 1
  turn = "RED"
  board = generate_board(n = n_cards, lang = lang, c = coloured_cards, k = k_cards)
  cards_remaining = {"RED": coloured_cards + 1, "BLUE": coloured_cards}

  if verbose:
      intro = prompt_colors["n"] + "GAME PARAMETERS:" + prompt_colors["endcolor"] + "\n" + \
      f"language = {lang}, \n" + \
      f"number of cards = {n_cards}, \n" + \
      f"number of killer cards = {k_cards}, \n" + \
      "\n" + \
      prompt_colors["RED"] + "RED TEAM PARAMETERS:" + prompt_colors["endcolor"] + "\n" + \
      "Red team starts firts, " + \
      f"number of red cards = {cards_remaining['RED']}, \n" + \
      f"red model = {red_model}, \n" + \
      f"number of red agents = {red_agents}, \n" + \
      "\n" + \
      prompt_colors["BLUE"] + "BLUE TEAM PARAMETERS:" + prompt_colors["endcolor"] + "\n" + \
      f"number of blue cards = {cards_remaining['BLUE']}, \n" + \
      f"blue model = {blue_model}, \n" + \
      f"number of blue agents = {blue_agents}, \n"

      if lang != "english":
        intro = translator(lang, intro)

      print("-----------------------------------")
      print(intro)
      print("-----------------------------------")

  while True:
    result = play_turn(lang, turn, board, cards_remaining, k_cards, red_agents if turn == "RED" else blue_agents, red_model if turn == "RED" else blue_model, verbose)
    if result["endgame"]:
      return (result["win"], result["reason"], r)
    else:
      cards_remaining = result["rc"]
      board = result["b"]
      r += 1
      turn = "BLUE" if turn == "RED" else "RED"


In [36]:
#@title testing
play_game(lang = "italian", n_cards = 10, coloured_cards = 3, k_cards = 1, verbose = True)

-----------------------------------
PARAMETRI DEL GIOCO
lingua = italiano
numero di carte = 10
numero di carte killer = 1

PARAMETRI DEL TEAM ROSSO
Il team rosso inizia per primo numero di carte rosse = 4
modello rosso = gpt-4o-mini
numero di agenti rossi = 3

PARAMETRI DEL TEAM BLU
numero di carte blu = 3
modello blu = gpt-4o-mini
numero di agenti blu = 3
-----------------------------------
NARRATOR: RED team turn, 

the actual board is: 

| LUNA (BLUE) | MONTAGNA (BLUE) | AMERICA (KILLER) | FRANCIA (RED) | JET (WHITE) |
| BOOM (RED) | GIOCO (RED) | DINOSAURO (RED) | CODICE (WHITE) | RADICE (BLUE) |




 RED SPYMASTER:clue: ESPLORAZIONE, number: 3



RED guesser 1 said: Siamo di fronte al tabellone e abbiamo ricevuto la parola chiave "ESPLORAZIONE" con il numero 3. Questo significa che dobbiamo trovare 3 parole che possono essere associate a questo concetto. Ecco come possiamo affrontare il problema:

### Analisi delle parole
1. **LUNA**: Spazio ed esplorazione scientifica, può richia

('BLUE', 'cards finished', 2)